In [ ]:
pip install supervision==0.26.1 --force-reinstall --no-deps # 

In [ ]:
https://github.com/pyresearch/notebooks

In [ ]:
https://colab.research.google.com/github/pyresearch/notebooks/blob/main/notebook/how_to_finetune_rf_detr_on_detection_dataset.ipynb

In [1]:
!pwd

/home/oleg/projects/ML/RT-DETR-RF-DETR


In [8]:
from roboflow import Roboflow
rf = Roboflow(api_key="1xSkA1WLYlGLbcHdbDla")
project = rf.workspace("uav-qnoms").project("uav-wqshy")
version = project.version(6)
dataset = version.download("yolov11")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to UAV-6 in yolov11:: 100%|█████████████████| 16446/16446 [00:00<00:00, 17411.19it/s]


In [4]:
from ultralytics import RTDETR
import requests
import json

# Посмотрим какие модели доступны
url = "https://github.com/ultralytics/assets/releases/download/v8.3.0/"
models_to_try = [
    'rtdetr-l.pt',
    'rtdetr-x.pt', 
    'rtdetr-m.pt',
    'rtdetr-s.pt'
]

for model_name in models_to_try:
    try:
        model = RTDETR(model_name)
        print(f"✅ Доступна: {model_name}")
        break
    except:
        print(f"❌ Недоступна: {model_name}")

✅ Доступна: rtdetr-l.pt


In [6]:
from ultralytics import RTDETR

# Load a COCO-pretrained RT-DETR-l model
model = RTDETR("rtdetr-l.pt")
#model = RTDETR('rtdetr-m.pt')  # ⬅️ Быстрее
# model = RTDETR('rtdetr-s.pt')  # ⬅️ Намного быстрее

# Display model information (optional)
model.info()

rt-detr-l summary: 449 layers, 32,970,476 parameters, 0 gradients, 108.3 GFLOPs


(449, 32970476, 0, 108.3437056)

In [2]:
from ultralytics import RTDETR

results = model.train(data="/home/oleg/projects/ML/RT-DETR-RF-DETR/UAV-6_YOLO11/data.yaml",
                    epochs=1,
                    batch=8,     # Средний batch  
                    imgsz=640, #640,
                    half=True,           # ⬅️ FP16 precision (осторожно!) 
                    profile=True,        # ⬅️ Покажет что тормозит  
                    #compile=True,  # ⬅️ УСКОРЯЕТ В 1.5-2 РАЗА!
                    workers=16,  
                    cache=True,   # Кеширование
                    amp=True) # Mixed precision

New https://pypi.org/project/ultralytics/8.3.223 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.221 🚀 Python-3.11.14 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4070, 12282MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/home/oleg/projects/ML/RT-DETR-RF-DETR/UAV-6_YOLO11/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=1, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=True, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=rtdetr-l.pt, momentum=0.937, mosaic=1.0, multi_scale=Fal

/home/oleg/miniconda3/envs/tensflow/lib/python3.11/site-packages/torch/autograd/graph.py:825: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at ../aten/src/ATen/Context.cpp:91.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


        1/1      6.62G     0.5277     0.9445     0.3543         15        640: 100% ━━━━━━━━━━━━ 938/938 2.6it/s 5:56<0.4s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 30/30 5.9it/s 5.1s0.1s
                   all        477        548      0.873       0.75       0.83      0.477

1 epochs completed in 0.101 hours.
Optimizer stripped from /home/oleg/projects/ML/RT-DETR-RF-DETR/runs/detect/train13/weights/last.pt, 66.2MB
Optimizer stripped from /home/oleg/projects/ML/RT-DETR-RF-DETR/runs/detect/train13/weights/best.pt, 66.2MB

Validating /home/oleg/projects/ML/RT-DETR-RF-DETR/runs/detect/train13/weights/best.pt...
Ultralytics 8.3.221 🚀 Python-3.11.14 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4070, 12282MiB)
rt-detr-l summary: 302 layers, 31,985,795 parameters, 0 gradients, 103.4 GFLOPs
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 30/30 6.8it/s 4.4s0.1s
        

In [ ]:
# Run inference with the RT-DETR-l model on the 'bus.jpg' image
results = model("path/to/bus.jpg")